# Synthetic Experiments

## Sample synthetic data

In [ ]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np

import choice_learn
from python.data import SyntheticDataGenerator
from choice_learn.basket_models import Trip, TripDataset

## Sample purchased baskets

In [ ]:
items_nests = {0:[0, 1, 2],
1: [3, 4, 5],
2: [6],
3: [7]}

nests_interactions = [["", "compl", "neutral", "neutral"],
["compl", "", "neutral", "neutral"],
["neutral", "neutral", "", "neutral"],
["neutral", "neutral", "neutral", ""]]

data_gen = SyntheticDataGenerator(items_nest=items_nests, nests_interactions=nests_interactions)

In [ ]:
dataset = data_gen.generate_dataset(n_baskets=1000)

In [ ]:
trip_list = []
for basket in dataset:
    trip_list.append(Trip(purchases=basket, prices=np.zeros((8, )), assortment=0))

trip_dataset = TripDataset(trips=trip_list, available_items=np.ones((1, 8)))

## Modelling 

In [ ]:
from choice_learn.basket_models import AleaCarta

In [ ]:
latent_sizes = {"preferences": 6, "price": 3, "season": 3}
n_negative_samples = 2
optimizer = "adam"
lr = 1e-3
epochs = 150
batch_size = 32

model = AleaCarta(
    item_intercept=False,
    price_effects=False,
    seasonal_effects=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

model.instantiate(n_items=8, n_stores=2)

In [ ]:
history = model.fit(trip_dataset)

In [ ]:
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training of Shopper")
plt.show()

## Results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
mask = np.ones((8,8), dtype=bool)
res = []
for i in range(8):
        r = model.compute_batch_utility(item_batch=np.array(list(range(8))),
                basket_batch=np.array([[i] for _ in range(8)]) ,
                store_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                week_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                price_batch=np.array([[0, 0, 0, 0, 0, 0] for _ in range(8)]))
        m = np.ones(8)
        m[i] = 0
        den = np.exp(r) * m
        r = den / den.sum()
        # r = np.concatenate([tf.nn.softmax(np.concatenate([r[:i], r[i+1:]]))[:i], [.0], tf.nn.softmax(np.concatenate([r[:i], r[i+1:]]))[i:]])
        res.append(r)
        mask[i][i] = False

res = np.stack(res)
mask = np.ma.masked_where(mask, res)

axes.set_xticks([], [])
axes.set_yticks([], [])
im = axes.imshow(np.stack(res), cmap="Spectral", alpha=0.99, vmin=0, vmax=1)
axes.imshow(mask, cmap=mpl.colors.ListedColormap(['white']), alpha=1)

cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.69])
fig.colorbar(im, cax=cbar_ax)
axes.set_title("Estimated Conditional Probabilities")